# STAT 207 Group Lab Assignment 14 - [10 total points]

## Regularization Models & Selecting a Tuning Parameter

<hr>

## <u>Lab Grading</u>:

Should we grade your submission?  If not, write the netID of the submission to be graded.  (Note: We will only grade one assignment per group, and we'll pick the first one that says we should grade that submission.  We will assign the same grade to all team members.)

*For example*, you might respond: **grade this submission** or **my submission is under netID jdeeke**

grade this submission

If you said **my submission is under netID** above, we will not read any more of your lab submission.

If you said **grade this submission** above, who worked with you on this submission?  Write both their **names** and **netIDs**.  

Also, discuss and record your summer plans.

JoonWoo Ha (ha47) -> go back to Korea

Soobin Jang (soobinj2) -> also go back to Korea

## <u>Purpose</u>:
You should work in groups of 2-3 on this report (not working in groups without permission will result in a point deduction). The purpose of this group lab assignment is to fit a regularized model and assess its performance on new data.
<hr>

## <u>Assignment Instructions</u>:

### Group Roles

Suggested and specified roles are provided below: 

#### Groups of 2

* **Driver**: This student will type the report.  While typing the report, you may be the one who is selecting the functions to apply to the data.
* **Navigator**: This student will guide the process of answering the question.  Specific ways to help may include: outlining the general steps needed to solve a question (providing the overview), locating examples within the course notes, and reviewing each line of code as it is typed.

#### Groups of 3

* **Driver**: This student will type the report.  They may also be the one to select the functions to apply to the data.
* **Navigator**: This student will guide the process of answering the question.  They may select the general approach to answering the question and/or a few steps to be completed along the way. 
* **Communicator**: This student will review the report (as it is typed) to ensure that it is clear and concise.  This student may also locate relevant examples within the course notes that may help complete the assignment.

<hr>

### Imports

In [1]:
#Run this
import pandas as pd                    # imports pandas and calls the imported version 'pd'
import matplotlib.pyplot as plt        # imports the package and calls it 'plt'
import seaborn as sns                  # imports the seaborn package with the imported name 'sns'
sns.set()  

## Case Study: Avoiding Underwater Weighing

We will look at data collected on 252 males in 1985.  In particular, we will consider a measure of the percent of body fat in these males, as measured by Siri's equation (`siri`).  This particular measure is obtained through an underwater weighing technique and is quite extensive and resource-intensive.  We would like to consider alternative (and easier to capture information) to get roughly the same information that the `siri` variable currently contains.  We have other body measures for these males available, including:

- **age**: Age (yrs)
- **weight**: Weight (lbs)
- **height**: Height (inches)
- **adipos**: BMI index
- **neck**: Neck circumference (cm)
- **chest**: Chest circumference (cm)
- **abdom**: Abdomen circumference (cm)
- **hip**: Hip circumference (cm)
- **dthigh**: Thigh circumference (cm)
- **knee**: Knee circumference (cm)
- **ankle**: Ankle circumference (cm)
- **biceps**: Extended bicepts circumference (cm)
- **forearm**: Forearm circumference (cm)
- **wrist**: Wrist circumference (cm)

**We will use all variables in the data as predictors except for the siri, brozek, density, and free (fat free weight) variables**, since these four variables are challenging measurements to obtain and therefore might not be available for a different male that is not included in our current data.

The code cell below will read in the data for you.  Be sure to run the cell. 

In [2]:
df = pd.read_csv('fat.csv')

df

,brozek,siri,density,age,weight,height,adipos,free,neck,chest,abdom,hip,thigh,knee,ankle,biceps,forearm,wrist
0,12.6,12.3,1.0708,23,154.25,67.75,23.7,134.9,36.2,93.1,85.2,94.5,59.0,37.3,21.9,32.0,27.4,17.1
1,6.9,6.1,1.0853,22,173.25,72.25,23.4,161.3,38.5,93.6,83.0,98.7,58.7,37.3,23.4,30.5,28.9,18.2
2,24.6,25.3,1.0414,22,154.00,66.25,24.7,116.0,34.0,95.8,87.9,99.2,59.6,38.9,24.0,28.8,25.2,16.6
3,10.9,10.4,1.0751,26,184.75,72.25,24.9,164.7,37.4,101.8,86.4,101.2,60.1,37.3,22.8,32.4,29.4,18.2
4,27.8,28.7,1.0340,24,184.25,71.25,25.6,133.1,34.4,97.3,100.0,101.9,63.2,42.2,24.0,32.2,27.7,17.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247,11.5,11.0,1.0736,70,134.25,67.00,21.1,118.9,34.9,89.2,83.6,88.8,49.6,34.8,21.5,25.6,25.7,18.5
248,32.3,33.6,1.0236,72,201.00,69.75,29.1,136.1,40.9,108.5,105.0,104.5,59.6,40.8,23.2,35.2,28.6,20.1
249,28.3,29.3,1.0328,72,186.75,66.00,30.2,133.9,38.9,111.1,111.5,101.7,60.3,37.3,21.5,31.3,27.2,18.0
250,25.3,26.0,1.0399,72,190.75,70.50,27.0,142.6,38.9,108.3,101.3,97.8,56.0,41.6,22.7,30.5,29.4,19.8


### 1. [1 point] Select a Regularization Technique 

We work for a company that is measuring the proportion of body fat for customers in order to design suits that can fit well for a customer that is ordering the suit over the internet.  We know that asking these customers to perform many body measurements will be burdensome for the customers, so we'd like to streamline the process by only asking the customer to provide a few key measurements.  We can then use our models to select an appropriately cut and tailored suit for the customer.

**a)** For this situation, what would our primary purpose of fitting the model be?  In other words, are we concerned about making predictions or understanding structures? 

In this situation, our primary purpose is to make accurate predictions. We aim to predict a customer's body fat percentage using only a few body measurements. Our goal is not to understand the underlying structure or relationships between variables, but rather to minimize prediction error in a practical setting.



**b)** We would like to use a regularization model for our fitted model.  What regularization technique would you suggest should be used based on the optimal design for the company and customer?

Considering your company's goals and your customers' convenience, it makes sense to use Lasso regression. Lasso automatically performs variable selection by zeroing out the coefficients of unimportant variables, helping you make accurate predictions while asking for minimal measurements from your customers. These characteristics are effective in simplifying models and improving customer experience.

### 2. [2.5 points] Prepare the Data

**a)** Split your data into training and test data.  Use a random state (you can choose your random state), and set aside 20% of your data for the test data.

In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split

df = df.dropna()

X = df.drop(columns=['brozek'])
y = df['brozek']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

**b)** We know that regularization models require some preparation of the data before the model can be fit.  Scale your $X$ explanatory variables for this model.

*Note:* You should perform this scaling in two stages.  You should both fit a scaler and transform your training data in the first stage.  In the second stage, you should adjust your test data by applying the scaler trained by your training data to your test data.

In [6]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)

X_test_scaled = scaler.transform(X_test)

**c)** Prepare your y variable for the model.

In [8]:
import numpy as np

y_train_array = np.array(y_train)
y_test_array = np.array(y_test)

### 3. [3 points] Fitting a Regularized Model

**a)** We'd like to fit a model using the regularization technique suggested in Question 1.  Start by fitting your model with a $\lambda$ of 2 to your training data.

In [10]:
from sklearn.linear_model import Lasso


lasso_model = Lasso(alpha=2)
lasso_model.fit(X_train_scaled, y_train_array)


coefficients = lasso_model.coef_

num_zero_coeffs = sum(coefficients == 0)
print("Number of slopes equal to 0:", num_zero_coeffs)

selected_features = X.columns[coefficients != 0]
print("Variables retained in the model:", list(selected_features))


Number of slopes equal to 0: 16
Variables retained in the model: ['siri']


**b)** How many of your slopes were equal to 0 for this model?  Which variables would you suggest should be retained in the model?

The Lasso regression analysis showed that a total of 16 variables had a coefficient of zero, meaning that they were deemed insignificant to the prediction and were removed from the model. The only variable with a non-zero coefficient was SIRI, and this variable was retained in the model because it was determined to be a significant contributor to the prediction. Therefore, it is appropriate to retain only the siri variable in the model.

**c)** Fit another version of your regularized model to your training data, but this time using a $\lambda$ of 0.5.

In [12]:
lasso_model_05 = Lasso(alpha=0.5)
lasso_model_05.fit(X_train_scaled, y_train_array)

coefficients_05 = lasso_model_05.coef_

num_zero_coeffs_05 = sum(coefficients_05 == 0)
print("Number of slopes equal to 0:", num_zero_coeffs_05)

selected_features_05 = X.columns[coefficients_05 != 0]
print("Variables retained in the model:", list(selected_features_05))


Number of slopes equal to 0: 16
Variables retained in the model: ['siri']


**d)** How many of your slopes were equal to 0 for this model?  Which variables would you suggest should be retained from this version of the model?

In the Lasso regression analysis with lambda = 0.5, a total of 16 variables have a coefficient of zero, which means that many variables have been removed, the same as in the previous model. The variable ‘siri’ is the only variable that has a non-zero coefficient and has been retained in the model. Therefore, it is appropriate to retain ‘siri’ in this version of the model as it is the only variable that is important for prediction.

### 4. [2 points] Comparing Model Results

**a)** Apply each of your two models fit from **3a** and **3c** to your test set.  Calculate the $R^2$ on your test set for each of your models.

In [13]:
from sklearn.metrics import r2_score

y_pred_2 = lasso_model.predict(X_test_scaled)
y_pred_05 = lasso_model_05.predict(X_test_scaled)

r2_2 = r2_score(y_test_array, y_pred_2)
r2_05 = r2_score(y_test_array, y_pred_05)

print("Test R² for model with lambda = 2:", r2_2)
print("Test R² for model with lambda = 0.5:", r2_05)


Test R² for model with lambda = 2: 0.9353999173825431
Test R² for model with lambda = 0.5: 0.9958635677243792


**b)** Which $\lambda$ would you suggest based on your test data?

When compared based on the R^2 value of the test data, the model with λ = 0.5 shows a high explanatory power of 0.9959, which is better than the R^2 of the λ = 2 model, which is 0.9354. Therefore, based on the performance of the test data, it is more appropriate to choose the model with λ = 0.5.

**c)** Compare the results to another group (or two) in your lab.  Did your models from **3** include the same predictor variables?  Did you pick the same preferred $\lambda$ from our two choices in **4b**?  Did you achieve the same $R^2$ values on your test data from **4a**?  Did you have the same training/test data split from **2a**?

When we compared our results with other groups, we found that in some cases they chose different predictor variables than our model, and some groups reported that they obtained models that retained additional variables besides ‘siri’. These differences were likely due to different values of the normalization strength λ or different splits of the training/test data.

Also, while some groups preferred the model with λ = 2, most groups tended to favor λ = 0.5, as it produced higher R² values. Our model had a very high test R^2 of 0.9959, but it was slightly different from the other groups because different data splits can lead to different R^2 values.

### 5. [1.5 points] A Limitation of the Results

Return to your selection of a regularization technique in Question 1 and recall that there are two primary purposes or benefits to using a regularization model.  Assess if there is still a concern in your model that the other regularization model approach could address.  Explain.

In [14]:
from sklearn.linear_model import Ridge, Lasso
from sklearn.metrics import r2_score

lasso = Lasso(alpha=0.5)
lasso.fit(X_train_scaled, y_train_array)
lasso_r2 = r2_score(y_test_array, lasso.predict(X_test_scaled))
print("Lasso R²:", lasso_r2)

ridge = Ridge(alpha=0.5)
ridge.fit(X_train_scaled, y_train_array)
ridge_r2 = r2_score(y_test_array, ridge.predict(X_test_scaled))
print("Ridge R²:", ridge_r2)

Lasso R²: 0.9958635677243792
Ridge R²: 0.9995568312355361


Ridge had higher predictive power while keeping all variables, suggesting that while Lasso is effective at selecting variables, it may lose information by removing some of the highly correlated variables. Ridge, on the other hand, keeps variables with multicollinearity together, which may provide more reliable coefficient estimates and generalization performance.